In [1]:
# dann_with_svm_keras_loader.py

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from functions import ReverseLayerF

from datasets import load_dataset
from keras.datasets import mnist
from keras.utils import to_categorical
from PIL import Image

# Load MNIST-M dataset
print("Loading MNIST-M from HuggingFace")
ds = load_dataset("Mike0307/MNIST-M")
train_data = ds['train']
test_data = ds['test']

X_train_m = np.array([np.array(image.resize((28, 28))) for image in train_data['image']])
y_train_m = np.array(train_data['label'])
X_test_m = np.array([np.array(image.resize((28, 28))) for image in test_data['image']])
y_test_m = np.array(test_data['label'])

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Preprocess
X_train = np.repeat(np.expand_dims(X_train.astype(np.float32) / 255, -1), 3, axis=-1)
X_test = np.repeat(np.expand_dims(X_test.astype(np.float32) / 255, -1), 3, axis=-1)
X_train_m = X_train_m.astype(np.float32) / 255
X_test_m = X_test_m.astype(np.float32) / 255
y_train_oh = to_categorical(y_train)
y_test_oh = to_categorical(y_test)
y_train_m_oh = to_categorical(y_train_m)
y_test_m_oh = to_categorical(y_test_m)

print('MNIST:', X_train.shape, y_train_oh.shape)
print('MNIST-M:', X_train_m.shape, y_train_m_oh.shape)

# Convert to PyTorch tensors
def to_tensor_dataset(x, y):
    x = torch.tensor(x.transpose((0, 3, 1, 2)), dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)
    return TensorDataset(x, y.argmax(dim=1))

dataset_source = to_tensor_dataset(X_train, y_train_oh)
dataset_target = to_tensor_dataset(X_train_m, y_train_m_oh)
dataloader_source = DataLoader(dataset_source, batch_size=128, shuffle=True)
dataloader_target = DataLoader(dataset_target, batch_size=128, shuffle=True)

# CNN model with domain adaptation
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.feature = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            nn.ReLU(True),
            nn.Conv2d(64, 50, kernel_size=5),
            nn.BatchNorm2d(50),
            nn.Dropout2d(),
            nn.MaxPool2d(2),
            nn.ReLU(True)
        )

        self.class_classifier = nn.Sequential(
            nn.Linear(50 * 4 * 4, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(100, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(True),
            nn.Linear(100, 10),
            nn.LogSoftmax(dim=1)
        )

        self.domain_classifier = nn.Sequential(
            nn.Linear(50 * 4 * 4, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(True),
            nn.Linear(100, 2),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, input_data, alpha):
        feature = self.feature(input_data)
        feature = feature.view(-1, 50 * 4 * 4)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)
        return class_output, domain_output

    def extract_features(self, input_data):
        with torch.no_grad():
            f = self.feature(input_data)
            return f.view(-1, 50 * 4 * 4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train DANN
model = CNNModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_class = nn.NLLLoss()
loss_domain = nn.NLLLoss()

for epoch in range(5):
    data_source_iter = iter(dataloader_source)
    data_target_iter = iter(dataloader_target)
    len_dataloader = min(len(data_source_iter), len(data_target_iter))

    for i in range(len_dataloader):
        p = float(i + epoch * len_dataloader) / (5 * len_dataloader)
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        s_img, s_label = next(data_source_iter)
        t_img, _ = next(data_target_iter)
        s_img, s_label = s_img.to(device), s_label.to(device)
        t_img = t_img.to(device)

        model.zero_grad()
        batch_size = s_img.size(0)
        domain_label = torch.zeros(batch_size).long().to(device)
        class_output, domain_output = model(s_img, alpha)
        loss_s_label = loss_class(class_output, s_label)
        loss_s_domain = loss_domain(domain_output, domain_label)

        batch_size = t_img.size(0)
        domain_label = torch.ones(batch_size).long().to(device)
        _, domain_output = model(t_img, alpha)
        loss_t_domain = loss_domain(domain_output, domain_label)

        loss = loss_s_label + loss_s_domain + loss_t_domain
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch} | Step {i} | Loss: {loss.item():.4f}")

from torch.utils.data import TensorDataset, DataLoader

# Convert test data to tensors
X_test_tensor = torch.tensor(X_test_m.transpose((0, 3, 1, 2)), dtype=torch.float32)
y_test_tensor = torch.tensor(np.argmax(y_test_m_oh, axis=1), dtype=torch.long)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


from sklearn.metrics import classification_report

all_preds = []
all_labels = []

model.eval()
for imgs, labels in test_loader:
    imgs = imgs.to(device)
    with torch.no_grad():
        class_output, _ = model(imgs, alpha=0)
    preds = class_output.argmax(dim=1).cpu().numpy()
    all_preds.extend(preds)
    all_labels.extend(labels.numpy())

# Print classification report
print("Classification Report on MNIST-M:")
print(classification_report(all_labels, all_preds, digits=4))


c:\Users\anshb\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading MNIST-M from HuggingFace
MNIST: (60000, 28, 28, 3) (60000, 10)
MNIST-M: (59001, 28, 28, 3) (59001, 10)
Epoch 0 | Step 0 | Loss: 3.8178
Epoch 0 | Step 1 | Loss: 3.7548
Epoch 0 | Step 2 | Loss: 3.6082
Epoch 0 | Step 3 | Loss: 3.5327
Epoch 0 | Step 4 | Loss: 3.5086
Epoch 0 | Step 5 | Loss: 3.3640
Epoch 0 | Step 6 | Loss: 3.3641
Epoch 0 | Step 7 | Loss: 3.3340
Epoch 0 | Step 8 | Loss: 3.2079
Epoch 0 | Step 9 | Loss: 3.2623
Epoch 0 | Step 10 | Loss: 3.2197
Epoch 0 | Step 11 | Loss: 3.1063
Epoch 0 | Step 12 | Loss: 3.0948
Epoch 0 | Step 13 | Loss: 3.1867
Epoch 0 | Step 14 | Loss: 3.0106
Epoch 0 | Step 15 | Loss: 3.1064
Epoch 0 | Step 16 | Loss: 2.8881
Epoch 0 | Step 17 | Loss: 2.9139
Epoch 0 | Step 18 | Loss: 2.8100
Epoch 0 | Step 19 | Loss: 2.8390
Epoch 0 | Step 20 | Loss: 2.8300
Epoch 0 | Step 21 | Loss: 2.8313
Epoch 0 | Step 22 | Loss: 2.8213
Epoch 0 | Step 23 | Loss: 2.7214
Epoch 0 | Step 24 | Loss: 2.6160
Epoch 0 | Step 25 | Loss: 2.6466
Epoch 0 | Step 26 | Loss: 2.6427
Epoch 0 